In [151]:
# Enable jupyter's asy io to avoid loop issue
import nest_asyncio
nest_asyncio.apply()

In [152]:
# init janusgraph graph database
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.structure.graph import Graph
from gremlin_python import statics
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.process.traversal import *
import asyncio
statics.load_statics(globals()) 
graph = Graph()
connection = DriverRemoteConnection('ws://localhost:8182/gremlin', 'g')
# The connection should be closed on shut down to close open connections with connection.close()
g = graph.traversal().withRemote(connection)
# Reuse 'g' across the application

# g.tx().commit()

In [153]:
import hnswlib
import numpy as np
import sys
import time
from sklearn.neighbors import NearestNeighbors

np.set_printoptions(threshold=np.inf)

In [154]:
def compute_and_store_distances(k, exact, distance_function, relationship_name):
        start = time.time()
        data, data_labels = get_transaction_vectors()
        print("Time to get vectors:", time.time() - start)
        start = time.time()

        if exact:
            ann_labels, ann_distances = compute_knn(data, data_labels, k, distance_function)
        else:
            ann_labels, ann_distances = compute_ann(data, data_labels, k, distance_function)

        print("the ann_labels is", ann_labels)
        print("the ann_distances is", ann_distances)


        print("Time to compute nearest neighbors:", time.time() - start)
        start = time.time()

        store_ann(data_labels, ann_labels, ann_distances, relationship_name)
        print("Time to store nearest neighbors:", time.time() - start)
        print("done")
        return ann_labels, ann_distances

def compute_ann(data, data_labels, k, distance_function):
        dim = len(data[0])
        num_elements = len(data_labels)
        # Declaring index
        p = hnswlib.Index(space=distance_function, dim=dim)  # possible options for ditance_formula are l2, cosine or ip
        # Initing index - the maximum number of elements should be known beforehand
        p.init_index(max_elements=num_elements, ef_construction=800, M=200)
        # Element insertion (can be called several times):
        p.add_items(data, data_labels)
        # Controlling the recall by setting ef:
        p.set_ef(800)  # ef should always be > k
        # Query dataset, k - number of closest elements (returns 2 numpy arrays)
        labels, distances = p.knn_query(data, k = k)

        return labels, distances

def compute_knn(data, data_labels, k, distance_function):
        pre_processed_data = [np.array(item) for item in data]
        nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=distance_function, n_jobs=-1).fit(pre_processed_data)
        knn_distances, knn_labels = nbrs.kneighbors(pre_processed_data)
        distances = knn_distances
        labels = [[data_labels[element] for element in item] for item in knn_labels]
        return labels, distances

def get_transaction_vectors():
    # add limit() to test this function to save test time
    # list_of_transaction_query = g.V().values("transactionId", "vector").limit(10).toList()

    list_of_transaction_query = g.V().values("transactionId", "vector").toList()
    data = []
    data_labels = []
    j = 0
    for i in list_of_transaction_query:
        j += 1   
        if isinstance(i, str):
            dataFloatList = []
            dataStringList = i.split(",")
            for item in dataStringList:
                l = float(item.replace("[", "").replace("]", ""))
                dataFloatList.append(l)
            data.append(dataFloatList)    
        else:
            data_labels.append(i)
        if j % 1000 == 0:
                print(j//2, "nodes processed") 

    return data, data_labels


def store_ann(data_labels, ann_labels, ann_distances, label): 
        # data_labels: the txc_id of source node list
        # ann_labels: the txc_id target node list[nparray]
        # ann_distances: the value of edge from "source node" to "target node"
        # label: the lable of edge
        
        # remove the edge if exists
        print("remove the edge if exists")
        g.E().hasLabel(label).drop().iterate()
        print("add the edge")

        # add the edge
        i = 0 #  the numbers of sourceNode    
        j = 0 #  the numbers of targetNode    
        for sourceNodeTransactionId in data_labels:
            sourceNode = g.V().has("transactionId", sourceNodeTransactionId).next()
            print("the sourceNode id is: ", sourceNode)
            for targetNodeTransactionId in ann_labels[i]:
                targetNode = g.V().has("transactionId", str(targetNodeTransactionId)).next()
                print("the targetNode id is: ", targetNode)
                for edgeWeight in ann_distances[i]:
                    print("the edgeWeight is: ", edgeWeight)
                    g.addE(label).property("weight", str(edgeWeight)).from_(sourceNode).to(targetNode).next()
                    j +=1
                i +=1
            print("the numbers of sourceNode and the numbers of targetNode is:", i, j)
            if (i >= len(data_labels)):
                break
                

In [155]:
if __name__ == '__main__':
    distance_formula_value = "l2" #'mahalanobis' for exact
    #relationship_name_value = "DISTANT_FROM_EXACT"
    relationship_name_value = "DISTANT_FROM"
    compute_and_store_distances(25, False, distance_formula_value, relationship_name_value)
    # Uncomment this line to calculate exact NNs, but it will take a lot of time!
    #distance_formula_value = 'mahalanobis'
    # relationship_name_value = "DISTANT_FROM_EXACT"
    # analyzer.compute_and_store_distances(25, True, distance_formula_value, relationship_name_value)
    # uncomment this line to close the graph database connection
    # connection.close()

500 nodes processed
1000 nodes processed
Time to get vectors: 1.4839661121368408
the ann_labels is [[   2   60  917  234  262  210  243  943  988  500  183  743  891   73
   357  101   81  603  155  443  818  254  883  968  767]
 [   4  604  366   13   57  270  343  140  792  231  873  842  765  671
   847  957  193  798   94  172  180  284  383  992  646]
 [  11   20  470  619  259  867  505  266  982  704  196  855  673  432
   506  748  220  442  361  650  651  328   85  188   82]
 [  13  343  366  604   57    4  873  231  792  270  671  140  847  842
   193  957  765  172  383  798   94  180  532  284  123]
 [  18  287  616  900  628  448  550  353  969  717  726  348  446  618
   153  743  787  913   10  591  254  358  507   35   36]
 [  20   11  505  700  470  585  259  952  619  748  204  994  966  101
    32  744  415  931  369  883   73  243  596  679  510]
 [  22   63  675  753  356  186  435  503  835  177  581  375  450   34
    95  273  941  453  833  316  326  431  314  7

the ann_distances is [[0.00000000e+00 9.54601765e-01 1.58326697e+00 2.64352512e+00
  2.78490686e+00 3.03637815e+00 3.28749967e+00 4.87131453e+00
  5.44642067e+00 5.56995487e+00 5.85370207e+00 6.20964575e+00
  6.23726606e+00 6.24510765e+00 6.26160526e+00 6.28464031e+00
  6.81641912e+00 6.85001516e+00 7.05012035e+00 7.10871124e+00
  7.19557190e+00 7.26482534e+00 7.31565762e+00 7.35694122e+00
  7.53974533e+00]
 [0.00000000e+00 2.95525856e+01 3.55962410e+01 3.83872299e+01
  4.02705994e+01 4.44536896e+01 4.51115074e+01 5.46206970e+01
  5.49207840e+01 6.23795128e+01 7.40688934e+01 9.82068481e+01
  1.04399017e+02 1.12019753e+02 1.14002518e+02 1.15119843e+02
  1.40579483e+02 1.57663910e+02 1.74588272e+02 1.92726837e+02
  2.04918228e+02 2.16972626e+02 2.37019348e+02 2.39518570e+02
  2.95113007e+02]
 [0.00000000e+00 9.10937786e+00 1.70071793e+01 1.72411785e+01
  2.02325974e+01 2.13981781e+01 2.18230476e+01 2.42598209e+01
  2.52097626e+01 2.56515045e+01 2.57435799e+01 2.60424747e+01
  2.63672504e

add the edge
the sourceNode id is:  v[4136]
the targetNode id is:  v[4136]
the edgeWeight is:  0.0
the edgeWeight is:  0.95460176
the edgeWeight is:  1.583267
the edgeWeight is:  2.6435251
the edgeWeight is:  2.7849069
the edgeWeight is:  3.0363781
the edgeWeight is:  3.2874997
the edgeWeight is:  4.8713145
the edgeWeight is:  5.4464207
the edgeWeight is:  5.569955
the edgeWeight is:  5.853702
the edgeWeight is:  6.2096457
the edgeWeight is:  6.237266
the edgeWeight is:  6.2451077
the edgeWeight is:  6.2616053
the edgeWeight is:  6.2846403
the edgeWeight is:  6.816419
the edgeWeight is:  6.850015
the edgeWeight is:  7.0501204
the edgeWeight is:  7.1087112
the edgeWeight is:  7.195572
the edgeWeight is:  7.2648253
the edgeWeight is:  7.3156576
the edgeWeight is:  7.356941
the edgeWeight is:  7.5397453
the targetNode id is:  v[49216]
the edgeWeight is:  0.0
the edgeWeight is:  29.552586
the edgeWeight is:  35.59624
the edgeWeight is:  38.38723
the edgeWeight is:  40.2706
the edgeWeight i

the edgeWeight is:  13.38748
the edgeWeight is:  13.425509
the edgeWeight is:  13.635366
the edgeWeight is:  13.6533785
the targetNode id is:  v[233664]
the edgeWeight is:  0.0
the edgeWeight is:  1.5030954
the edgeWeight is:  2.8432984
the edgeWeight is:  4.548071
the edgeWeight is:  5.563981
the edgeWeight is:  6.8157597
the edgeWeight is:  7.0490026
the edgeWeight is:  8.14822
the edgeWeight is:  9.888177
the edgeWeight is:  10.42254
the edgeWeight is:  10.739493
the edgeWeight is:  11.752712
the edgeWeight is:  11.757989
the edgeWeight is:  12.668065
the edgeWeight is:  14.530544
the edgeWeight is:  15.243753
the edgeWeight is:  15.260122
the edgeWeight is:  15.417908
the edgeWeight is:  15.510467
the edgeWeight is:  15.834121
the edgeWeight is:  15.925977
the edgeWeight is:  16.702604
the edgeWeight is:  16.736355
the edgeWeight is:  17.276913
the edgeWeight is:  17.635098
the targetNode id is:  v[385144]
the edgeWeight is:  0.0
the edgeWeight is:  7.280393
the edgeWeight is:  16.

the edgeWeight is:  14.967325
the edgeWeight is:  15.110136
the edgeWeight is:  15.535913
the targetNode id is:  v[401632]
the edgeWeight is:  0.0
the edgeWeight is:  13.147643
the edgeWeight is:  14.145071
the edgeWeight is:  17.121843
the edgeWeight is:  20.43673
the edgeWeight is:  21.485693
the edgeWeight is:  23.058475
the edgeWeight is:  23.823956
the edgeWeight is:  24.06021
the edgeWeight is:  27.156532
the edgeWeight is:  29.100296
the edgeWeight is:  29.312687
the edgeWeight is:  30.176933
the edgeWeight is:  31.315264
the edgeWeight is:  34.66452
the edgeWeight is:  35.49228
the edgeWeight is:  35.497044
the edgeWeight is:  37.975605
the edgeWeight is:  38.754803
the edgeWeight is:  39.0567
the edgeWeight is:  39.512638
the edgeWeight is:  40.444054
the edgeWeight is:  41.00769
the edgeWeight is:  41.206898
the edgeWeight is:  41.642365
the targetNode id is:  v[675880]
the edgeWeight is:  0.0
the edgeWeight is:  5.3827496
the edgeWeight is:  16.228952
the edgeWeight is:  18.

...

the edgeWeight is:  8.556488
the edgeWeight is:  8.816618
the edgeWeight is:  9.232133
the edgeWeight is:  9.302643
the edgeWeight is:  9.611923
the edgeWeight is:  9.919666
the edgeWeight is:  10.122599
the edgeWeight is:  10.34439
the edgeWeight is:  10.632709
the edgeWeight is:  11.137449
the edgeWeight is:  11.154444
the edgeWeight is:  11.243195
the edgeWeight is:  11.672802
the edgeWeight is:  11.772522
the edgeWeight is:  11.782106
the edgeWeight is:  11.79376
the targetNode id is:  v[233592]
the edgeWeight is:  0.0
the edgeWeight is:  8.999513
the edgeWeight is:  9.675971
the edgeWeight is:  9.915581
the edgeWeight is:  14.022554
the edgeWeight is:  14.584839
the edgeWeight is:  15.68371
the edgeWeight is:  15.834669
the edgeWeight is:  17.11798
the edgeWeight is:  19.305958
the edgeWeight is:  21.019386
the edgeWeight is:  22.700184
the edgeWeight is:  23.236656
the edgeWeight is:  23.504131
the edgeWeight is:  24.45472
the edgeWeight is:  25.12236
the edgeWeight is:  25.81031

the edgeWeight is:  74.40323
the edgeWeight is:  91.792435
the edgeWeight is:  104.47075
the edgeWeight is:  107.15222
the edgeWeight is:  113.22244
the edgeWeight is:  113.27919
the targetNode id is:  v[32960]
the edgeWeight is:  0.0
the edgeWeight is:  15.072995
the edgeWeight is:  17.957384
the edgeWeight is:  18.387024
the edgeWeight is:  18.693502
the edgeWeight is:  18.955505
the edgeWeight is:  19.6668
the edgeWeight is:  19.915375
the edgeWeight is:  21.156036
the edgeWeight is:  22.348
the edgeWeight is:  23.724586
the edgeWeight is:  24.192669
the edgeWeight is:  24.684584
the edgeWeight is:  25.041515
the edgeWeight is:  25.862709
the edgeWeight is:  25.914833
the edgeWeight is:  26.565905
the edgeWeight is:  27.33859
the edgeWeight is:  28.051414
the edgeWeight is:  28.470276
the edgeWeight is:  29.466753
the edgeWeight is:  30.09996
the edgeWeight is:  30.523441
the edgeWeight is:  30.899574
the edgeWeight is:  30.98515
the targetNode id is:  v[176288]
the edgeWeight is:  

the edgeWeight is:  23.375359
the edgeWeight is:  23.72531
the edgeWeight is:  24.28476
the edgeWeight is:  24.429726
the edgeWeight is:  25.342201
the edgeWeight is:  28.060965
the edgeWeight is:  28.547714
the edgeWeight is:  28.630535
the edgeWeight is:  29.524693
the edgeWeight is:  29.595722
the numbers of sourceNode and the numbers of targetNode is: 975 24375
the sourceNode id is:  v[163880]
the targetNode id is:  v[364768]
the edgeWeight is:  0.0
the edgeWeight is:  19.376228
the edgeWeight is:  21.235044
the edgeWeight is:  22.727888
the edgeWeight is:  23.609028
the edgeWeight is:  23.766834
the edgeWeight is:  25.057259
the edgeWeight is:  25.096375
the edgeWeight is:  25.426369
the edgeWeight is:  27.63013
the edgeWeight is:  29.070395
the edgeWeight is:  29.640858
the edgeWeight is:  30.265808
the edgeWeight is:  30.523655
the edgeWeight is:  30.599564
the edgeWeight is:  30.945814
the edgeWeight is:  31.095879
the edgeWeight is:  31.132736
the edgeWeight is:  31.736519
the

the edgeWeight is:  41.089535
the edgeWeight is:  41.221252
the edgeWeight is:  41.222183
the edgeWeight is:  41.425056
the edgeWeight is:  42.035423
the edgeWeight is:  42.200806
the edgeWeight is:  43.470337
the edgeWeight is:  43.97952
the edgeWeight is:  44.286766
the targetNode id is:  v[98368]
the edgeWeight is:  0.0
the edgeWeight is:  8.519291
the edgeWeight is:  9.201057
the edgeWeight is:  10.932646
the edgeWeight is:  13.426427
the edgeWeight is:  15.326437
the edgeWeight is:  15.549287
the edgeWeight is:  16.973991
the edgeWeight is:  17.047253
the edgeWeight is:  17.89382
the edgeWeight is:  18.622082
the edgeWeight is:  18.66716
the edgeWeight is:  19.783642
the edgeWeight is:  20.176039
the edgeWeight is:  21.736824
the edgeWeight is:  22.227108
the edgeWeight is:  23.738644
the edgeWeight is:  26.684698
the edgeWeight is:  27.294153
the edgeWeight is:  27.802557
the edgeWeight is:  28.476994
the edgeWeight is:  29.352543
the edgeWeight is:  29.397163
the edgeWeight is: 

the edgeWeight is:  8.824432
the edgeWeight is:  9.139863
the edgeWeight is:  9.280159
the edgeWeight is:  9.916463
the targetNode id is:  v[311488]
the edgeWeight is:  0.0
the edgeWeight is:  31.495201
the edgeWeight is:  35.284946
the edgeWeight is:  35.28701
the edgeWeight is:  37.878044
the edgeWeight is:  37.89006
the edgeWeight is:  38.549194
the edgeWeight is:  42.676655
the edgeWeight is:  43.36217
the edgeWeight is:  43.421623
the edgeWeight is:  44.468742
the edgeWeight is:  45.54811
the edgeWeight is:  46.084633
the edgeWeight is:  46.359016
the edgeWeight is:  46.380688
the edgeWeight is:  46.392723
the edgeWeight is:  47.671513
the edgeWeight is:  48.070923
the edgeWeight is:  48.336338
the edgeWeight is:  49.284084
the edgeWeight is:  49.399036
the edgeWeight is:  50.291504
the edgeWeight is:  50.655514
the edgeWeight is:  50.825863
the edgeWeight is:  50.929726
the targetNode id is:  v[221320]
the edgeWeight is:  0.0
the edgeWeight is:  10.607439
the edgeWeight is:  11.5

In [156]:
%%graph_notebook_config
{
  "host": "localhost",
  "port": 8182,
  "ssl": false,
  "gremlin": {
    "traversal_source": "g"
  }
}

set notebook config to:
{
  "host": "localhost",
  "port": 8182,
  "ssl": false,
  "sparql": {
    "path": ""
  },
  "gremlin": {
    "traversal_source": "g"
  }
}


In [157]:
%%gremlin -p v,oute,inv
g.V().outE("DISTANT_FROM").inV().path().by(id).by(label).limit(1000)